In [1]:
import os
import gc
import sys
import glob
import time
import math
import numpy as np
import uproot
import pandas
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import tensorflow as tf
import keras
import keras.backend as K
from keras import regularizers
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten, \
                                    Concatenate, PReLU, TimeDistributed, LSTM, Masking
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger
#from keras_tqdm import TQDMNotebookCallback

sys.path.insert(0, "../../python")
from common import *
from DataLoader import DataLoader, read_hdf_lock

Using TensorFlow backend.


In [2]:
class MaskedDense(Dense):
    def __init__(self, units, **kwargs):
        super(MaskedDense, self).__init__(units, **kwargs)
        
    def call(self, inputs, mask=None):
        base_out = super(MaskedDense, self).call(inputs)
        if mask is None:
            return base_out
        zeros = tf.zeros_like(base_out)
        return tf.where(mask, base_out, zeros)

In [2]:
class SafeModelCheckpoint(ModelCheckpoint):
    def __init__(self, filepath, **kwargs):
        super(SafeModelCheckpoint, self).__init__(filepath, **kwargs)
    
    def on_epoch_end(self, epoch, logs=None):
        read_hdf_lock.acquire()
        super(SafeModelCheckpoint, self).on_epoch_end(epoch, logs)
        read_hdf_lock.release()

In [4]:
class NetSetup:
    def __init__(self, activation, activation_shared_axes, dropout_rate, first_layer_size, last_layer_size, decay_factor,
                 kernel_regularizer, time_distributed):
        self.activation = activation
        self.activation_shared_axes = activation_shared_axes
        if activation == 'relu' or activation == 'PReLU' or activation == 'tanh':
            self.DropoutType = Dropout
            self.kernel_init = 'he_uniform'
            self.apply_batch_norm = True
        elif activation == 'selu':
            self.DropoutType = AlphaDropout
            self.kernel_init = 'lecun_normal'
            self.apply_batch_norm = False
        else:
            raise RuntimeError('Activation "{}" not supported.'.format(activation))
        self.dropout_rate = dropout_rate
        self.first_layer_size = first_layer_size
        self.last_layer_size = last_layer_size
        self.decay_factor = decay_factor
        self.kernel_regularizer = kernel_regularizer
        self.time_distributed = time_distributed
    
    def RecalcLayerSizes(self, n_input_features, width_factor, compression_factor, consider_dropout = True):
        drop_factor = 1 + self.dropout_rate if consider_dropout else 1
        self.first_layer_size = int(math.ceil(n_input_features * drop_factor * width_factor))
        self.last_layer_size = int(math.ceil(n_input_features * drop_factor * compression_factor))
    
def add_block_ending(net_setup, name_format, layer):
    if net_setup.apply_batch_norm:
        norm_layer = BatchNormalization(name=name_format.format('norm'))
        if net_setup.time_distributed:
            norm_layer = TimeDistributed(norm_layer, name=name_format.format('norm'))
        norm_layer = norm_layer(layer)
    else:
        norm_layer = layer
    if net_setup.activation == 'PReLU':
        activation_layer = PReLU(shared_axes=net_setup.activation_shared_axes,
                                 name=name_format.format('activation'))(norm_layer)
    else:
        activation_layer = Activation(net_setup.activation, name=name_format.format('activation'))(norm_layer)
    if net_setup.dropout_rate > 0:
        return net_setup.DropoutType(net_setup.dropout_rate, name=name_format.format('dropout'))(activation_layer)
    return activation_layer

def dense_block(prev_layer, kernel_size, net_setup, block_name, n):
    DenseType = MaskedDense if net_setup.time_distributed else Dense
    dense = DenseType(kernel_size, name="{}_dense_{}".format(block_name, n),
                      kernel_initializer=net_setup.kernel_init,
                      kernel_regularizer=net_setup.kernel_regularizer)
    if net_setup.time_distributed:
        dense = TimeDistributed(dense, name="{}_dense_{}".format(block_name, n))
    dense = dense(prev_layer)
    return add_block_ending(net_setup, '{}_{{}}_{}'.format(block_name, n), dense)

def reduce_n_features_1d(input_layer, net_setup, block_name):
    prev_layer = input_layer
    current_size = net_setup.first_layer_size
    n = 1
    while True:
        prev_layer = dense_block(prev_layer, current_size, net_setup, block_name, n)
        if current_size == net_setup.last_layer_size: break
        current_size = max(net_setup.last_layer_size, int(current_size / net_setup.decay_factor))
        n += 1
    return prev_layer

def dense_block_sequence(input_layer, net_setup, n_layers, block_name):
    prev_layer = input_layer
    current_size = net_setup.first_layer_size
    for n in range(n_layers):
        prev_layer = dense_block(prev_layer, current_size, net_setup, block_name, n+1)
    return prev_layer

def conv_block(prev_layer, filters, kernel_size, net_setup, block_name, n):
    conv = Conv2D(filters, kernel_size, name="{}_conv_{}".format(block_name, n),
                  kernel_initializer=net_setup.kernel_init)(prev_layer)
    return add_block_ending(net_setup, '{}_{{}}_{}'.format(block_name, n), conv)

def reduce_n_features_2d(input_layer, net_setup, block_name):
    conv_kernel=(1, 1)
    prev_layer = input_layer
    current_size = net_setup.first_layer_size
    n = 1
    while True:
        prev_layer = conv_block(prev_layer, current_size, conv_kernel, net_setup, block_name, n)
        if current_size == net_setup.last_layer_size: break
        current_size = max(net_setup.last_layer_size, int(current_size / net_setup.decay_factor))
        n += 1
    return prev_layer

def create_model(net_config):
    tau_net_setup = NetSetup('PReLU', None, 0.2, 128, 128, 1.4, None, False)
    comp_net_setup = NetSetup('PReLU', [1, 2], 0.2, 1024, 64, 1.6, None, False)
    #dense_net_setup = NetSetup('relu', 0, 512, 32, 1.4, keras.regularizers.l1(1e-5))
    dense_net_setup = NetSetup('PReLU', None, 0.2, 200, 64, 1.4, None, False)
            
    input_layers = []
    high_level_features = []

    if len(net_config.tau_branches) > 0:
        input_layer_tau = Input(name="input_tau", shape=(len(net_config.tau_branches),))
        input_layers.append(input_layer_tau)
        tau_net_setup.RecalcLayerSizes(len(net_config.tau_branches), 2, 1)
        processed_tau = reduce_n_features_1d(input_layer_tau, tau_net_setup, 'tau')
        #processed_tau = dense_block_sequence(input_layer_tau, tau_net_setup, 4, 'tau')
        high_level_features.append(processed_tau)
    
    for loc in net_config.cell_locations:
        reduced_inputs = []
        for comp_id in range(len(net_config.comp_names)):
            comp_name = net_config.comp_names[comp_id]
            n_comp_features = len(input_cell_external_branches) + len(net_config.comp_branches[comp_id])
            input_layer_comp = Input(name="input_{}_{}".format(loc, comp_name),
                                     shape=(n_cells_eta[loc], n_cells_phi[loc], n_comp_features))
            input_layers.append(input_layer_comp)
            comp_net_setup.RecalcLayerSizes(n_comp_features, 2, 1)
            #input_layer_comp_masked = Masking(name="input_{}_{}_masking".format(loc, comp_name))(input_layer_comp)
            #reduced_comp = dense_block_sequence(input_layer_comp_masked, comp_net_setup, 4, "{}_{}".format(loc, comp_name))
            #reduced_comp = reduce_n_features_1d(input_layer_comp_masked, comp_net_setup, "{}_{}".format(loc, comp_name))
            reduced_comp = reduce_n_features_2d(input_layer_comp, comp_net_setup, "{}_{}".format(loc, comp_name))
            reduced_inputs.append(reduced_comp)
        
        cell_output_size = 64
        if len(net_config.comp_names) > 1:
            conv_all_start = Concatenate(name="{}_cell_concat".format(loc), axis=3)(reduced_inputs)
            comp_net_setup.first_layer_size = conv_all_start.shape.as_list()[3]
            comp_net_setup.last_layer_size = 64
            prev_layer = reduce_n_features_2d(conv_all_start, comp_net_setup, "{}_all".format(loc))
        else:
            prev_layer = reduced_inputs[0]
        window_size = 3
        current_size = n_cells_eta[loc]
        n = 1
        while current_size > 1:
            win_size = min(current_size, window_size)
            prev_layer = conv_block(prev_layer, cell_output_size, (win_size, win_size), comp_net_setup,
                                    "{}_all_{}x{}".format(loc, win_size, win_size), n)
            n += 1
            current_size -= window_size - 1
            
        cells_flatten = Flatten(name="{}_cells_flatten".format(loc))(prev_layer)
        high_level_features.append(cells_flatten)
        
    if len(high_level_features) > 1:
        features_concat = Concatenate(name="features_concat", axis=1)(high_level_features)
    else:
        features_concat = high_level_features[0]
    if net_config.final:
        #print(features_concat.get_shape())
        #dense_net_setup.RecalcLayerSizes(128, 1, 0.5, False)
        #final_dense = reduce_n_features_1d(features_concat, dense_net_setup, 'final')
        final_dense = dense_block_sequence(features_concat, dense_net_setup, 4, 'final')
        output_layer = Dense(n_outputs, name="final_dense_last",
                             kernel_initializer=dense_net_setup.kernel_init)(final_dense)

    else:
        final_dense = dense_block(features_concat, 1024, dense_net_setup,
                                  'tmp_{}'.format(net_config.name), 1)
        output_layer = Dense(n_outputs, name="tmp_{}_dense_last".format(net_config.name),
                             kernel_initializer=dense_net_setup.kernel_init)(final_dense)
    softmax_output = Activation("softmax", name="main_output")(output_layer)
    
    model = Model(input_layers, softmax_output, name="DeepTau2017v2")
    return model

In [3]:
def compile_model(model, learning_rate):
    opt = keras.optimizers.Adam(lr=learning_rate)
    #opt = keras.optimizers.Nadam(lr=learning_rate)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    metrics = [
        "accuracy", TauLosses.tau_crossentropy, TauLosses.tau_crossentropy_v2,
        TauLosses.Le, TauLosses.Lmu, TauLosses.Ljet,
        TauLosses.He, TauLosses.Hmu, TauLosses.Htau, TauLosses.Hjet,
        TauLosses.Hcat_e, TauLosses.Hcat_mu, TauLosses.Hcat_jet, TauLosses.Hbin,
        TauLosses.Hcat_eInv, TauLosses.Hcat_muInv, TauLosses.Hcat_jetInv,
        TauLosses.Fe, TauLosses.Fmu, TauLosses.Fjet,
    ]
    model.compile(loss=TauLosses.tau_crossentropy_v2, optimizer=opt, metrics=metrics, weighted_metrics=metrics)

In [6]:
TauLosses.SetSFs(1, 1, 3, 1)
model_name = "DeepTau2017v2p5"
model = create_model(netConf_full)
compile_model(model, 1e-3)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_inner_egamma (InputLayer) (None, 11, 11, 86)   0                                            
__________________________________________________________________________________________________
input_inner_muon (InputLayer)   (None, 11, 11, 64)   0                                            
__________________________________________________________________________________________________
input_inner_hadrons (InputLayer (None, 11, 11, 38)   0                                            
__________________________________________________________________________________________________
input_outer_egamma (I

In [7]:
def GetNumberOfTrainableParams(model, exclude_prefix = 'tmp'):
    count = 0
    for w in model.trainable_weights:
        if not w.name.startswith(exclude_prefix):
            count += K.count_params(w)
    return count
GetNumberOfTrainableParams(model)

969753

In [4]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [5]:
class TimeCheckpoint(Callback):
    def __init__(self, time_interval, file_name_prefix):
        self.time_interval = time_interval
        self.file_name_prefix = file_name_prefix
        self.initial_time = time.time()
        self.last_check_time = self.initial_time
    
    def on_batch_end(self, batch, logs=None):
        if batch % 100 != 0: return
        current_time = time.time()
        delta_t = current_time - self.last_check_time
        if delta_t >= self.time_interval:
            abs_delta_t_h = (current_time - self.initial_time) / 60. / 60.
            read_hdf_lock.acquire()
            self.model.save('{}_b{}_{:.1f}h.h5'.format(self.file_name_prefix, batch, abs_delta_t_h))
            read_hdf_lock.release()
            self.last_check_time = current_time

In [6]:
def run_training(train_suffix, model_name, data_loader, epoch, n_epochs):

    train_name = '%s_%s' % (model_name, train_suffix)
    
    cb_acc = []
    for acc_name in ["acc", "weighted_acc"]:
        cb_acc.append(SafeModelCheckpoint("%s_acc.hdf5" % train_name, monitor="val_%s" % acc_name, save_best_only=True,
                                          save_weights_only=False, mode="max", verbose=1))
    
    losses_names = [ "loss" ]
    for w_suffix in [ "", "weighted_" ]:
        for l_name in [ "tau_crossentropy", "tau_crossentropy_v2", "Le", "Lmu", "Ljet", "He", "Hmu", "Htau", "Hjet",
                        "Hcat_e", "Hcat_mu", "Hcat_jet", "Hbin" ]:
            losses_names.append(w_suffix + l_name)
    cb_losses = []
    for loss_name in losses_names:
        cb_losses.append(SafeModelCheckpoint("%s_%s.hdf5" % (train_name, loss_name), monitor="val_%s" % loss_name,
                                             save_best_only=True, save_weights_only=False, mode="min", verbose=1))

    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    time_checkpoint = TimeCheckpoint(8*60*60, '{}_historic'.format(train_name))
    #pbar = TQDMNotebookCallback(leave_outer=True, show_outer=True, leave_inner = True)
    callbacks = [time_checkpoint, csv_log, *cb_acc, *cb_losses]
    fit_hist = model.fit_generator(data_loader.generator(True), validation_data=data_loader.generator(False),
                                   steps_per_epoch=data_loader.steps_per_epoch, validation_steps=data_loader.validation_steps,
                                   callbacks=callbacks, epochs=n_epochs, initial_epoch=epoch, verbose=1)

    model.save("%s_final.hdf5" % train_name)
    return fit_hist

In [7]:
loader = DataLoader('N:/tau-ml/tuples-v2-t3/training/part_*.h5', netConf_full, 500, 10000, validation_size=6100000,
                    max_queue_size=40, n_passes=-1, return_grid=True)

print(loader.total_size, loader.data_size, loader.validation_size)

72491602 66391602 6100000


In [ ]:
fit_hist = run_training('step{}'.format(1), model_name, loader, 0, 4)

Instructions for updating:
Use tf.cast instead.
Epoch 1/4
118159/132784 [=========================>....] - ETA: 2:00:54 - loss: 0.2279 - acc: 0.8102 - tau_crossentropy: 0.1677 - tau_crossentropy_v2: 0.2248 - Le: 0.0778 - Lmu: 0.0146 - Ljet: 0.1592 - He: 0.1148 - Hmu: 0.1911 - Htau: 0.0350 - Hjet: 0.1579 - weighted_acc: 0.8125 - weighted_tau_crossentropy: 0.1510 - weighted_tau_crossentropy_v2: 0.2279 - weighted_Le: 0.0888 - weighted_Lmu: 0.0222 - weighted_Ljet: 0.1156 - weighted_He: 0.1331 - weighted_Hmu: 0.2107 - weighted_Htau: 0.0547 - weighted_Hjet: 0.0927

In [8]:
TauLosses.SetSFs(1, 1, 3, 1)
model_name = "DeepTau2017v2p5"
model = LoadModel('DeepTau2017v2p5_step1_loss.hdf5')
opt_weights = K.batch_get_value(model.optimizer.weights)
compile_model(model, 1e-3)
model._make_train_function()
model.optimizer.set_weights(opt_weights)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [9]:
fit_hist = run_training('step{}'.format(1), model_name, loader, 1, 4)

Epoch 2/4
132784/132784 [==============================] - 69107s 520ms/step - loss: 0.1909 - acc: 0.8798 - tau_crossentropy: 0.1382 - tau_crossentropy_v2: 0.1889 - Le: 0.0650 - Lmu: 0.0104 - Ljet: 0.1319 - He: 0.0941 - Hmu: 0.0869 - Htau: 0.0314 - Hjet: 0.1236 - Hcat_e: 0.0516 - Hcat_mu: 0.0048 - Hcat_jet: 0.1101 - Hbin: 0.0076 - Hcat_eInv: 0.0425 - Hcat_muInv: 0.0821 - Hcat_jetInv: 0.0135 - HbinInv: 0.1398 - weighted_acc: 0.8664 - weighted_tau_crossentropy: 0.1262 - weighted_tau_crossentropy_v2: 0.1909 - weighted_Le: 0.0788 - weighted_Lmu: 0.0175 - weighted_Ljet: 0.0930 - weighted_He: 0.1065 - weighted_Hmu: 0.1400 - weighted_Htau: 0.0468 - weighted_Hjet: 0.0726 - weighted_Hcat_e: 0.0568 - weighted_Hcat_mu: 0.0059 - weighted_Hcat_jet: 0.0637 - weighted_Hbin: 0.0065 - weighted_Hcat_eInv: 0.0497 - weighted_Hcat_muInv: 0.1341 - weighted_Hcat_jetInv: 0.0089 - weighted_HbinInv: 0.1012 - val_loss: 0.1604 - val_acc: 0.9041 - val_tau_crossentropy: 0.1295 - val_tau_crossentropy_v2: 0.1718 - va

KeyboardInterrupt: 

In [8]:
TauLosses.SetSFs(1, 3, 5, 1)
model_name = "DeepTau2017v2p5"
model = LoadModel('DeepTau2017v2p5_step1_epoch1_loss.hdf5')
opt_weights = K.batch_get_value(model.optimizer.weights)
compile_model(model, 1e-3)
model._make_train_function()
model.optimizer.set_weights(opt_weights)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [9]:
fit_hist = run_training('step{}'.format(1), model_name, loader, 2, 4)

Epoch 3/4
132784/132784 [==============================] - 69336s 522ms/step - loss: 0.2991 - acc: 0.8002 - tau_crossentropy: 0.0864 - tau_crossentropy_v2: 0.2995 - Le: 0.0616 - Lmu: 0.0182 - Ljet: 0.0998 - He: 0.2664 - Hmu: 0.0248 - Htau: 0.0681 - Hjet: 0.2023 - Hcat_e: 0.0289 - Hcat_mu: 0.0016 - Hcat_jet: 0.0620 - Hbin: 0.0600 - Hcat_eInv: 0.2376 - Hcat_muInv: 0.0232 - Hcat_jetInv: 0.1403 - Fe: 0.0034 - Fmu: 5.5431e-04 - Fjet: 0.0058 - weighted_acc: 0.7881 - weighted_tau_crossentropy: 0.1028 - weighted_tau_crossentropy_v2: 0.2991 - weighted_Le: 0.0815 - weighted_Lmu: 0.0309 - weighted_Ljet: 0.0826 - weighted_He: 0.2743 - weighted_Hmu: 0.0460 - weighted_Htau: 0.0876 - weighted_Hjet: 0.1547 - weighted_Hcat_e: 0.0313 - weighted_Hcat_mu: 0.0018 - weighted_Hcat_jet: 0.0371 - weighted_Hbin: 0.0444 - weighted_Hcat_eInv: 0.2430 - weighted_Hcat_muInv: 0.0442 - weighted_Hcat_jetInv: 0.1176 - weighted_Fe: 0.0042 - weighted_Fmu: 4.0140e-04 - weighted_Fjet: 0.0034 - val_loss: 0.2386 - val_acc: 0.


Epoch 00004: val_Hmu did not improve from 0.01345

Epoch 00004: val_Htau did not improve from 0.04773

Epoch 00004: val_Hjet did not improve from 0.22314

Epoch 00004: val_Hcat_e improved from 0.02627 to 0.02564, saving model to DeepTau2017v2p5_step1_Hcat_e.hdf5

Epoch 00004: val_Hcat_mu improved from 0.00148 to 0.00144, saving model to DeepTau2017v2p5_step1_Hcat_mu.hdf5

Epoch 00004: val_Hcat_jet did not improve from 0.07057

Epoch 00004: val_Hbin improved from 0.06283 to 0.06029, saving model to DeepTau2017v2p5_step1_Hbin.hdf5

Epoch 00004: val_weighted_tau_crossentropy improved from 0.07946 to 0.07915, saving model to DeepTau2017v2p5_step1_weighted_tau_crossentropy.hdf5

Epoch 00004: val_weighted_tau_crossentropy_v2 improved from 0.23858 to 0.23430, saving model to DeepTau2017v2p5_step1_weighted_tau_crossentropy_v2.hdf5

Epoch 00004: val_weighted_Le improved from 0.06285 to 0.06059, saving model to DeepTau2017v2p5_step1_weighted_Le.hdf5

Epoch 00004: val_weighted_Lmu did not improv